Use cases are being developed to demonstrate the usage of PicoQuant and for use in benchmarking and profiling activities.

## Quantum Fourier transfrom use case

The Quantum Fourier Transform (QFT) is at the heart of many quantum algorithms. For this use case a simple implementation from [https://arxiv.org/abs/quant-ph/0201067](https://arxiv.org/abs/quant-ph/0201067) has been implemented. Here the gate count grows approximately with the square of the number of qubits. A methond called `create_qft_circuit` can be used to construct a qiskit circuit object for this circuit for a given number of qubits. For example to create a QFT circuit for 3 qubits one can use:

In [ ]:
using PicoQuant

qft_circ = create_qft_circuit(3)

qft_circ.draw()

We can convert this circuit to a tensor network circuit using the `convert_qiskit_circ_to_network` method as 

In [ ]:
InteractiveBackend()
tn = convert_qiskit_circ_to_network(qft_circ)
plot(tn)

We then add an all zero input state and use a full wavefunction contraction plan to contract the network.

In [ ]:
add_input!(tn, "000")
full_wavefunction_contraction!(tn, "vector")
psi = load_tensor_data(backend, :result)

This gives the final transformed state. We can verify this is the correct answer by carrying out an inverst fourier transform on the input vector.

In [ ]:
using FFTW

function get_ft_with_ifft(input_state)
    output_state = ifft(input_state)
    output_state / sqrt(sum(output_state .* conj(output_state)))
end

input_state = zeros(ComplexF64, 8)
input_state[1] = 1.
get_ft_with_ifft(input_state)

We can test with a less trivial input state by using a state preparation circuit to prepare a less trivial state. 

In [ ]:
prep_circ = create_simple_preparation_circuit(3, 2)
prep_circ.draw()

We combine this with the QFT circuit to apply the QFT to the prepared state and then run the full wavefunction contraction to get the final state

In [ ]:
combined_circ = prep_circ.compose(qft_circ)
tn = convert_qiskit_circ_to_network(combined_circ)
add_input!(tn, "000")
psi = full_wavefunction_contraction!(tn, "vector")

To check that the output is correct we get the output of just the state preparation circuit and apply the IFFT with appropriate normalization (using the get_ft_with_ifft defined above).

In [ ]:
tn = convert_qiskit_circ_to_network(prep_circ)
add_input!(tn, "000")
psi_input = full_wavefunction_contraction!(tn, "vector")
ref_output = get_ft_with_ifft(psi_input)

We see that this matches the state we got above (doesn't exactly at the minute due to a re-ordering issue which should be resolved shortly).

## Grover search use case